<a href="https://colab.research.google.com/github/bhgtankita/MYWORK/blob/master/LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import re
import string
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.text import Tokenizer

In [0]:
# load ascii text and covert to lowercase
filename = "gdrive/My Drive/Gutenberg/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [8]:
tokenizer = Tokenizer(char_level=True)
  
tr = str.maketrans("", "", string.punctuation)
data = raw_text.translate(tr)
  
# basic cleanup
corpus = [line for line in data.lower().split("\n") if line.strip() != '']
# print(corpus)

# tokenization	
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(total_words)

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
#   print(token_list)
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
#     print(input_sequences)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = np_utils.to_categorical(label, num_classes=total_words)

28


In [0]:
X = np.expand_dims(predictors, axis=-1)

In [10]:
print(X.shape)
print(label.shape)

(129978, 72, 1)
(129978, 28)


In [0]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(label.shape[1], activation='softmax'))

In [14]:
# define the checkpoint
filepath="gdrive/My Drive/Gutenberg/weights/weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0726 16:35:10.814195 139860711278464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [16]:
model.fit(X, label, epochs=100, batch_size=128, callbacks=callbacks_list)

W0726 16:35:43.201711 139860711278464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 16:35:43.207754 139860711278464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 16:35:43.222575 139860711278464 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0726 16:35:43.245902 139860711278464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backe

Epoch 1/100
129978/129978 [==============================] - 286s 2ms/step - loss: 2.5305

Epoch 00001: loss improved from inf to 2.53047, saving model to gdrive/My Drive/Gutenberg/weights/weights-01-2.5305.hdf5
Epoch 2/100
129978/129978 [==============================] - 281s 2ms/step - loss: 2.1196

Epoch 00002: loss improved from 2.53047 to 2.11962, saving model to gdrive/My Drive/Gutenberg/weights/weights-02-2.1196.hdf5
Epoch 3/100
129978/129978 [==============================] - 279s 2ms/step - loss: 1.9479

Epoch 00003: loss improved from 2.11962 to 1.94786, saving model to gdrive/My Drive/Gutenberg/weights/weights-03-1.9479.hdf5
Epoch 4/100
129978/129978 [==============================] - 281s 2ms/step - loss: 1.8442

Epoch 00004: loss improved from 1.94786 to 1.84424, saving model to gdrive/My Drive/Gutenberg/weights/weights-04-1.8442.hdf5
Epoch 5/100
129978/129978 [==============================] - 281s 2ms/step - loss: 1.7735

Epoch 00005: loss improved from 1.84424 to 1.7735

# Final loss at 100 epoch was 1.1009

In [0]:
def generate_text(seed_text, next_words, max_sequence_len):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    test = np.expand_dims(token_list, axis=-1)
    predicted = model.predict_classes(test, verbose=0)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

In [18]:
print(generate_text("be no mistake about it: it was neither more nor less than a pig, and she felt that it would be quit be no mistake about it: it was neither more nor less than a pig, and she felt that it would be quit", 500, max_sequence_len))

be no mistake about it: it was neither more nor less than a pig, and she felt that it would be quit be no mistake about it: it was neither more nor less than a pig, and she felt that it would be quit h n a r a   a t h n g a h e d a r a t h n e y o n a r a   a r a t h n g o n a r a   a r a t h n g i n a r a   a t h e y a t o t h n g i n a r a   a t h e y a t o n a t h n   a r a t h n g o n a r a   a r a t h n g o n a r a   a r a t h n g i n a r a   a t h e y a t o t h n g i n a r a   a t h e y a t o n a t h n   a r a t h n g o n a r a   a r a t h n g o n a r a   a r a t h n g i n a r a   a t h e y a t o t h n g i n a r a   a t h e y a t o n a t h n   a r a t h n g o n a r a   a r a t h n g o n a r a   a r a t h n g i n a r a   a t h e y a t o t h n g i n a r a   a t h e y a t o n a t h n   a r a t h n g o n a r a   a r a t h n g o n a r a   a r a t h n g i n a r a   a t h e y a t o t h n g i n a r a   a t h e y a t o n a t h n   a r a t h n g o n a r a   a r a t h n g o n a r a   a r a 